# Agents Intensive - Capstone Project - Monisha Shah

**Kaggle 5-day Agents course!**

Creating agents to get the recipes for sugar free deserts along with Ingredients.  

Using a Sequential Agents to retrieve the recipe for the desert and where to buy the ingredients from.  


### Install dependencies google agent development kit library

The Kaggle Notebooks environment includes a pre-installed version of the [google-adk](https://google.github.io/adk-docs/) library for Python and its required dependencies, so you don't need to install additional packages in this notebook.

To install and use ADK in your own Python development environment outside of this course, you can do so by running:

```
pip install google-adk
```

### Configure your Gemini API Key

This notebook uses the [Gemini API](https://ai.google.dev/gemini-api/docs), which requires authentication.

**1. Get your API key**

If you don't have one already, create an [API key in Google AI Studio](https://aistudio.google.com/app/api-keys).

**2. Add the key to Kaggle Secrets**

Next, you will need to add your API key to your Kaggle Notebook as a Kaggle User Secret.

1. In the top menu bar of the notebook editor, select `Add-ons` then `Secrets`.
2. Create a new secret with the label `GOOGLE_API_KEY`.
3. Paste your API key into the "Value" field and click "Save".
4. Ensure that the checkbox next to `GOOGLE_API_KEY` is selected so that the secret is attached to the notebook.

**3. Authenticate in the notebook**

Run the cell below to complete authentication.

In [1]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Gemini API key setup complete.


### Import ADK components

Now, import the specific components you'll need from the Agent Development Kit and the Generative AI library. This keeps your code organized and ensures we have access to the necessary building blocks.

In [2]:
from google.adk.agents import Agent, LlmAgent, SequentialAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search
from google.genai import types



from google.adk.tools.mcp_tool.mcp_toolset import McpToolset
from google.adk.tools.tool_context import ToolContext
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters

from google.adk.apps.app import App, ResumabilityConfig
from google.adk.tools.function_tool import FunctionTool
from google.adk.sessions import InMemorySessionService

print("✅ ADK components imported successfully.")

✅ ADK components imported successfully.


### Helper functions

We'll define some helper functions. If you are running this outside the Kaggle environment, you don't need to do this.

In [3]:
# Define helper functions that will be reused throughout the notebook

from IPython.core.display import display, HTML
from jupyter_server.serverapp import list_running_servers


# Gets the proxied URL in the Kaggle Notebooks environment
def get_adk_proxy_url():
    PROXY_HOST = "https://kkb-production.jupyter-proxy.kaggle.net"
    ADK_PORT = "8000"

    servers = list(list_running_servers())
    if not servers:
        raise Exception("No running Jupyter servers found.")

    baseURL = servers[0]["base_url"]

    try:
        path_parts = baseURL.split("/")
        kernel = path_parts[2]
        token = path_parts[3]
    except IndexError:
        raise Exception(f"Could not parse kernel/token from base URL: {baseURL}")

    url_prefix = f"/k/{kernel}/{token}/proxy/proxy/{ADK_PORT}"
    url = f"{PROXY_HOST}{url_prefix}"

    styled_html = f"""
    <div style="padding: 15px; border: 2px solid #f0ad4e; border-radius: 8px; background-color: #fef9f0; margin: 20px 0;">
        <div style="font-family: sans-serif; margin-bottom: 12px; color: #333; font-size: 1.1em;">
            <strong>⚠️ IMPORTANT: Action Required</strong>
        </div>
        <div style="font-family: sans-serif; margin-bottom: 15px; color: #333; line-height: 1.5;">
            The ADK web UI is <strong>not running yet</strong>. You must start it in the next cell.
            <ol style="margin-top: 10px; padding-left: 20px;">
                <li style="margin-bottom: 5px;"><strong>Run the next cell</strong> (the one with <code>!adk web ...</code>) to start the ADK web UI.</li>
                <li style="margin-bottom: 5px;">Wait for that cell to show it is "Running" (it will not "complete").</li>
                <li>Once it's running, <strong>return to this button</strong> and click it to open the UI.</li>
            </ol>
            <em style="font-size: 0.9em; color: #555;">(If you click the button before running the next cell, you will get a 500 error.)</em>
        </div>
        <a href='{url}' target='_blank' style="
            display: inline-block; background-color: #1a73e8; color: white; padding: 10px 20px;
            text-decoration: none; border-radius: 25px; font-family: sans-serif; font-weight: 500;
            box-shadow: 0 2px 5px rgba(0,0,0,0.2); transition: all 0.2s ease;">
            Open ADK Web UI (after running cell below) ↗
        </a>
    </div>
    """

    display(HTML(styled_html))

    return url_prefix


print("✅ Helper functions defined.")

✅ Helper functions defined.


### Configure Retry Options

When working with LLMs, you may encounter transient errors like rate limits or temporary service unavailability. Retry options automatically handle these failures by retrying the request with exponential backoff.

In [4]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

---

## AI Agent with ADK

### What is an AI Agent?

You've probably used an LLM like Gemini before, where you give it a prompt and it gives you a text response.

`Prompt -> LLM -> Text`

An AI Agent takes this one step further. An agent can think, take actions, and observe the results of those actions to give you a better answer.

`Prompt -> Agent -> Thought -> Action -> Observation -> Final Answer`

In this notebook, we'll build an agent that can take the action of searching Google. Let's see the difference!

### Define your agent

Now, let's build our agent. We'll configure an `Agent` by setting its key properties, which tell it what to do and how to operate.

To learn more, check out the documentation related to [agents in ADK](https://google.github.io/adk-docs/agents/).

These are the main properties we'll set:

- **name** and **description**: A simple name and description to identify our agent.
- **model**: The specific LLM that will power the agent's reasoning. We'll use "gemini-2.5-flash-lite".
- **instruction**: The agent's guiding prompt. This tells the agent what its goal is and how to behave.
- **tools**: A list of [tools](https://google.github.io/adk-docs/tools/) that the agent can use. To start, we'll give it the `google_search` tool, which lets it find up-to-date information online.

**Agent that helps retrieve the sugar free recipes**

In [5]:
recipe_agent = Agent(
    name="recipe_assistant",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="A simple agent that gets you the recipes of all sugarfree deserts using dried dates, figs and raisins.",
    instruction="You are a helpful assistant. Use Google Search to get the recipe for the sugarfree dessert.",
    tools=[google_search],
    output_key="recipe_generated",  # The result will be stored with this key.
)

print("✅ Recipe Agent defined.")

✅ Recipe Agent defined.


**Ingredients Agent created to get all the ingredients only for the summary?**

In [6]:
ingredients_agent = Agent(
    name="ingredients_assistant",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="A simple agent that gets you the recipes of all sugarfree deserts using dried dates, figs and raisins.",
    instruction="Please only give me the ingredients for the recipe for the sugarfree dessert. {recipe_generated}",
    output_key="ingredients_generated",  # The result will be stored with this key.
)

print("✅ Ingredients Agent defined.")


✅ Ingredients Agent defined.


**Agent to retrieve the couple stores the ingredients are available at?******

In [7]:
store_suggestion_agent = Agent(
    name="ingredients_assistant",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="A simple agent that get you the stores to buy the ingredients from?",
    instruction="Please give me the top 2 stores where the ingredients can be bought?. {ingredients_generated}",
    output_key="stores_suggestion_generated",  # The result will be stored with this key.
)

print("✅ Stores Suggestion Agent defined.")


✅ Stores Suggestion Agent defined.


**Agent to get the cumulative pricing for the ingredients**

In [8]:
price_estimation_agent = Agent(
    name="price_estimation_assistant",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="A simple agent that gets you the prices of theingredients",
    instruction="Please give me the prices of all the ingredients can be bought from one store?. {ingredients_generated}",
    output_key="prices_suggestion_generated",  # The result will be stored with this key.
)

print("✅ Prices Suggestion Agent defined.")

✅ Prices Suggestion Agent defined.


**Sequential Agent created to get the recipe, the ingredients and stores suggestion agents in a sequential manner. 

System with three specialized agents:

recipe_agent - gets the recipe
ingredients_agent - gets the ingredients
stores suggestion agent - gets the stores the ingredients are available at 

In [9]:
root_agent = SequentialAgent(
    name="RecipeWithImagePipeline",
    sub_agents=[recipe_agent, ingredients_agent, store_suggestion_agent, price_estimation_agent],
)

print("✅ Sequential Agent created.")

✅ Sequential Agent created.


### Run your agent

Now it's time to bring your agent to life and send it a query. To do this, you need a [`Runner`](https://google.github.io/adk-docs/runtime/), which is the central component within ADK that acts as the orchestrator. It manages the conversation, sends our messages to the agent, and handles its responses.

**a. Create an `InMemoryRunner` and tell it to use our `root_agent`:**
**Using run_debug to send out prompt and get the answer**

In [10]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "Get me a sugar free recipe for pumpkin pie with ingredients and stores where the ingredients are available"
)


 ### Created new session: debug_session_id

User > Get me a sugar free recipe for pumpkin pie with ingredients and stores where the ingredients are available
recipe_assistant > Here is a sugar-free pumpkin pie recipe, sweetened naturally with dates and maple syrup, along with information on where to find the ingredients:

**Sugar-Free Pumpkin Pie**

This recipe offers a delicious and healthier alternative to traditional pumpkin pie, using natural sweeteners and real pumpkin.

**Yields:** 6-8 servings
**Prep time:** 45 minutes
**Cook time:** 40-50 minutes

**Ingredients:**

*   1 unbaked 9-inch pie crust (homemade or store-bought, ensure it's sugar-free if store-bought)
*   1/2 cup soft Medjool dates, pitted and chopped
*   1/4 cup pure maple syrup
*   1 1/2 cups pumpkin puree (ensure it's pure pumpkin, not pie filling)
*   1 teaspoon cinnamon
*   1 teaspoon ground ginger
*   1/8 teaspoon ground nutmeg
*   1/2 teaspoon salt
*   3 large eggs
*   1 cup milk (whole milk, almond milk, or c

👉 Note that we are using the Python Runner directly in this notebook. You can also run agents using ADK command-line tools such as `adk run`, `adk web`, or `adk api_server`. To learn more, check out the documentation related to [runtime in ADK](https://google.github.io/adk-docs/runtime/).

**b. Now you can call the `.run_debug()` method to send our prompt and get an answer.**

👉 This method abstracts the process of session creation and maintenance and is used in prototyping. We'll explore "what sessions are and how to create them" on Day 3.

In [11]:
response = await runner.run_debug(
    "Give me a recipe for Apple Pie?  List the ingredients out separately and stores showing availability?"
)


 ### Continue session: debug_session_id

User > Give me a recipe for Apple Pie?  List the ingredients out separately and stores showing availability?
recipe_assistant > Here is a recipe for a sugar-free apple pie, using dates and maple syrup for sweetness, along with ingredient availability:

**Sugar-Free Apple Pie**

This recipe offers a comforting and naturally sweetened apple pie, perfect for those avoiding refined sugars.

**Yields:** 6-8 servings
**Prep time:** 30 minutes
**Cook time:** 45-55 minutes

**Ingredients:**

**For the Filling:**

*   6-8 medium apples (such as Honeycrisp, Fuji, or Gala), peeled, cored, and sliced
*   1/2 cup soft Medjool dates, pitted and chopped
*   1/4 cup pure maple syrup
*   1 tablespoon lemon juice
*   1 teaspoon ground cinnamon
*   1/4 teaspoon ground nutmeg
*   1/8 teaspoon ground allspice (optional)
*   2 tablespoons cornstarch or arrowroot powder (for thickening)
*   1 tablespoon unsalted butter or coconut oil (optional, for richness)

**For t

You can see a summary of ADK and its available languages in the response.

### How does it work?

The agent performed a Google Search to get the latest information about ADK, and it knew to use this tool because:

1. The agent inspects and is aware of which tools it has available to use.
2. The agent's instructions specify the use of the search tool to get current information or if it is unsure of an answer.

The best way to see the full, detailed trace of the agent's thoughts and actions is in the **ADK web UI**, which we'll set up later in this notebook.

And we'll cover more detailed workflows for logging and observability later in the course.

### 🚀 Checking out other sugar free recipes 

In [12]:
response = await runner.run_debug("What's the recipe for Indian sweet Gulab Jamun?")


 ### Continue session: debug_session_id

User > What's the recipe for Indian sweet Gulab Jamun?
recipe_assistant > Here's a recipe for Gulab Jamun, a popular Indian sweet, along with ingredient information:

**Gulab Jamun Recipe**

Gulab Jamun are soft, spongy milk-solid-based balls, deep-fried and soaked in a fragrant sugar syrup. They are a quintessential Indian dessert, often served during festivals and celebrations. This recipe uses milk powder, which is more readily available than traditional khoya (milk solids).

**Yields:** Approximately 20-24 gulab jamuns
**Prep time:** 30 minutes (+ 10 minutes resting)
**Cook time:** 20-30 minutes (frying) + 2-3 hours (soaking)

**Ingredients:**

**For the Gulab Jamun Balls:**

*   1 ½ cups milk powder (whole milk powder recommended for best results)
*   ½ cup all-purpose flour (maida)
*   ½ teaspoon baking powder
*   ¼ teaspoon salt (optional, some recipes omit)
*   2 tablespoons ghee (clarified butter), melted, plus more for greasing finger

In [13]:
response = await runner.run_debug("What's the recipe for Indian sweet Gulab Jamun not using milk powder?")


 ### Continue session: debug_session_id

User > What's the recipe for Indian sweet Gulab Jamun not using milk powder?
recipe_assistant > Here's a recipe for Gulab Jamun made without milk powder, using khoya (reduced milk solids) or ricotta cheese as the base. This method often results in a richer, more traditional flavor.

**Traditional Gulab Jamun (using Khoya or Ricotta)**

This recipe uses khoya, which is essentially evaporated milk solids. If khoya is not readily available, good quality whole milk ricotta cheese can be used as a substitute, though the texture and flavor will be slightly different.

**Yields:** Approximately 20-24 gulab jamuns
**Prep time:** 30 minutes
**Cook time:** 20-30 minutes (frying) + 2-3 hours (soaking)

**Ingredients:**

**For the Gulab Jamun Balls:**

*   1 cup khoya (mawa), crumbled or grated (see notes below for ricotta substitute)
*   ¼ cup all-purpose flour (maida)
*   1 tablespoon all-purpose flour (maida) for dusting
*   ¼ teaspoon baking soda (not 

---

## 💻 Section 3: Try the ADK Web Interface

### Overview

ADK includes a built-in web interface for interactively chatting with, testing, and debugging your agents.

<img width="1200" src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day1/adk-web-ui.gif" alt="ADK Web UI" />

To use the ADK web UI, you'll need to create an agent with Python files using the `adk create` command.

Run the command below to generate a `sample-agent` folder that contains all the necessary files, including `agent.py` for your code, an `.env` file with your API key pre-configured, and an `__init__.py` file:

In [14]:
!adk create sample-agent --model gemini-2.5-flash-lite --api_key $GOOGLE_API_KEY


Agent created in /kaggle/working/sample-agent:
- .env
- __init__.py
- agent.py



Get your custom URL to access the ADK web UI in the Kaggle Notebooks environment:

In [15]:
url_prefix = get_adk_proxy_url()

Now we can run ADK web:

In [ ]:
!adk web --url_prefix {url_prefix}

/usr/local/lib/python3.11/dist-packages/google/adk/cli/fast_api.py:130: UserWarning: [EXPERIMENTAL] InMemoryCredentialService: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  credential_service = InMemoryCredentialService()
/usr/local/lib/python3.11/dist-packages/google/adk/auth/credential_service/in_memory_credential_service.py:33: UserWarning: [EXPERIMENTAL] BaseCredentialService: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__()
INFO:     Started server process [93]
INFO:     Waiting for application startup.

+-----------------------------------------------------------------------------+
| ADK Web Server started                                                      |
|                                                                             |
| For local testing, access at http:/

Now you can access the ADK dev UI using the link above.

Once you open the link, you'll see the ADK web interface where you can ask your ADK agent questions.

Note: This sample agent does not have any tools enabled (like Google Search). It is a basic agent designed specifically to let you explore the UI features.

‼️ **IMPORTANT: DO NOT SHARE THE PROXY LINK** with anyone - treat it as sensitive data as it contains your authentication token in the URL.